In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def calculate_matrix(data, covariance=True, skip_missing=True):
    """
    """
    if not isinstance(data, (np.ndarray, pd.DataFrame)):
        data = np.array(data)
    if isinstance(data, np.ndarray):
        data = pd.DataFrame(data)

    if skip_missing:
        clean_data = data.dropna()
        if covariance:
            result = clean_data.cov()
        else:
            result = clean_data.corr()
    else:
        result = pairwise_calc(data, covariance=covariance)

    return result

def pairwise_calc(data, covariance=True):
    """
    """
    n = data.shape[1]
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            pairwise_data = data.iloc[:, [i, j]].dropna()
            if pairwise_data.empty:
                matrix[i, j] = np.nan
            elif covariance:
                matrix[i, j] = np.cov(pairwise_data.iloc[:, 0], pairwise_data.iloc[:, 1])[0, 1]
            else:
                matrix[i, j] = np.corrcoef(pairwise_data.iloc[:, 0], pairwise_data.iloc[:, 1])[0, 1]
    return pd.DataFrame(matrix, columns=data.columns, index=data.columns)


In [ ]:
def expW(m, λ):
    w = np.zeros(m)
    for i in range(m):
        w[i] = (1 - λ) * (λ ** (m - i - 1))
    w = w / w.sum()
    return w

def ewCovar(x, λ):
    m, n = x.shape
    w = expW(m, λ)
    weighted_mean = w @ x
    centered = x - weighted_mean
    sqrt_w = np.sqrt(w).reshape(-1, 1)
    xm = sqrt_w * centered
    cov = xm.T @ xm
    return cov

In [ ]:
def covariance_to_correlation(cov_matrix):

    std_dev = np.sqrt(np.diag(cov_matrix))
    std_diag = np.diag(1 / std_dev)
    corr_matrix = std_diag @ cov_matrix @ std_diag

    return corr_matrix

In [ ]:
def combined_weighted_cov_to_corr(data, lam_var, lam_cov):
    """
    计算结合不同指数加权的协方差矩阵，并将其标准化为相关性矩阵。

    参数:
        data (numpy.ndarray): 输入数据矩阵 (n_samples, n_features)。
        lam_var (float): 用于方差的指数加权因子。
        lam_cov (float): 用于协方差的指数加权因子。

    返回:
        numpy.ndarray: 标准化后的相关性矩阵。
    """
    def exp_weighted_cov(data, lam):
        """
        计算指数加权协方差矩阵。

        参数:
            data (numpy.ndarray): 输入数据矩阵。
            lam (float): 指数加权因子 (0 < lam <= 1)。

        返回:
            numpy.ndarray: 指数加权协方差矩阵。
        """
        n, m = data.shape
        weights = (1 - lam) * lam ** np.arange(n - 1, -1, -1)
        weights /= weights.sum()
        weighted_mean = np.average(data, axis=0, weights=weights)
        adjusted_data = np.sqrt(weights)[:, np.newaxis] * (data - weighted_mean)
        return adjusted_data.T @ adjusted_data

    # Step 1: 计算 lam_var 的指数加权协方差矩阵并提取标准差
    cov_matrix_var = exp_weighted_cov(data, lam_var)
    sd1 = np.sqrt(np.diag(cov_matrix_var))

    # Step 2: 计算 lam_cov 的指数加权协方差矩阵并提取标准差倒数
    cov_matrix_cov = exp_weighted_cov(data, lam_cov)
    sd = 1 / np.sqrt(np.diag(cov_matrix_cov))

    # Step 3: 标准化协方差矩阵
    diag_sd1 = np.diag(sd1)
    diag_sd = np.diag(sd)
    combined_corr_matrix = diag_sd1 @ diag_sd @ cov_matrix_cov @ diag_sd @ diag_sd1

    return combined_corr_matrix